# sdmlib

`sdmlib` is a simple Python library providing an implementation of SDM that is intended for use with `numpy` arrays. I wrote this library for personal use while writing my undergraduate thesis in Neuroscience.

# Overview

Sparse Distributed Memory (SDM) is a technique for simulating computer memory with significantly more storage than is actually possible. Modern computer use $64$-bit memory, but SDM allows us to simulate $1000$-bit memory (or more!) by distributing the operations of reading and writing over a small subset of randomly sampled addresses. 

# Computer Memory
You can locate a house on a street using, for example, a $3$-digit combination of the digits $0$ through $9$. This gives you $10^3 = 1000$ possible addresses. Maybe your friend Robin lives at address $831$.

Similarly, you can specify a location in *computer memory* using a 64-digit combination of the binary digits ("bits") $0$ and $1$. This gives you $2^{64} = 18,446,744,073,709,551,616$ possible memory addresses to choose from. (In reality we do not use all of these addresses, but the principle still holds). We use these addresses to store data. Like addresses, data are stored in the form of $0$s and $1$s, and are usually $64$ bits long (just like street addresses). 

# Motivation
SDM was originally developed by Pentti Kanerva as a mathematical model for human long-term memory. Kanerva noted that high-dimensional spaces have some properties in common with human long-term memory. For example, similar memories can be 'nearby' in the same sense that points in high-dimensional space can be 'nearby'. Furthermore, two random memories tend to be unrelated just like two random points in high-dimensional space tend to be far apart. For Kanerva, a sufficiently high number of dimensions begins at least in the hundreds, if not in the thousands.

The simplest high-dimensional space is the binary space $\{0, 1 \}^N$. This space has $N$ dimensions, where each dimension can *only* take on the value $0$ or $1$. On a $64$-bit computer, every possible address is in $\{0, 1 \}^{64}$.

This space is a good candidate for our mathematical model of human long-term memory because:
1. it is high-dimensional
2. it is "simple" (only two values)
3. it can hypothetically be implemented on a digital computer

# The Problem
The possibility of implementing a model of human memory on a computer is exciting. However, even modern computers that use $64$-bit addresses are not sufficiently 'high-dimensional' for the properties Kanerva was interested in. We are interested in having, for example, $1000$ bit memory addresses. Sadly, this would require $2^{1000}$ locations to store data in our computer and our universe only have $2^{265}$ atoms, so we will run out of atoms before we can construct such a computer.

# Solution to the Problem
To solve this problem, Kanerva suggests only implementing a small, randomly sampled number of *hard addresses* - actual locations in computer memory corresponding to a point in high-dimensional space. We use $M$ to denote the number of hard addresses to implement. $M$ is often small, like $10^6$ (small relative to $2^N$). 

If we randomly pick an address from $\{ 0, 1 \}^N$, we are extremely unlikely to pick an address that has a corresponding hard address. In fact, the odds are

$$
\frac{M}{2^N}
$$

which, if we take $M=10^6$ and $N=1000$ (common values used in SDM), the odds are approximately $9.3 \times 10^{-296}$.

Let's arrange our hard addresses in an $M \times N$ matrix where each row is a hard address. We can call this matrix $A$.

$$
A = \begin{bmatrix}
1 & 0 & 0 & 1 & 1 \dots & 1 \\
0 & 0 & 1 & 1 & 0 \dots & 1 \\
1 & 1 & 0 & 1 & 1 \dots & 1 \\
1 & 0 & 0 & 0 & 0 \dots & 1 \\
\vdots & & & & & \vdots \\
0 & 1 & 1 & 1 & 0 \dots & 1 \\
\end{bmatrix}
$$

Imagine trying to write to an $N$-bit address called $x$. It is unlikely that $x$ is a row of $A$. How, then, can we write to this address? We instead pick hard addresses that are 'close' to $x$. In this case, two addresses are close if the *Hamming distance* between them is below some threshold $d$.

The Hamming distance between two binary strings of equal length is the number of places where their bits do not match. For example, the Hamming distance between $1001 \; 1111$ and $1011 \; 1110$ is $2$.

We go through each row $i$ in $A$. If the Hamming distance between $x$ $A_i$ is smaller than $d$, then we consider that address to be 'nearby'.

We can make a vector $h$ of length $M$ to keep track of the Hamming distance between each row of $A$ and our desired address $x$. We can make a similar vector $y$ that is $1$ where $h < d$ and $0$ otherwise.

<img src="{{site.url}}/assets/sdm1.drawio.png">

To recap: we have a matrix $A$ of addresses. It has $M$ rows and $N$ columns, corresponding to the $M$ hard addresses and the $N$ bits of each address. Given an $N$-bit address $x$ that we would like to access, we construct a vector $y$. If a row $i$ of $A$ has a Hamming distance to $x$ that is smaller than $d$, we set $y_i$ to $1$ and otherwise set it to $0$. 

How do we write to memory? Let's say we have some data $w$ that we want to write. $w$ is a binary vector of length $U$. In normal computer memory, we pick a single address $x$ and overwrite the contents of that memory with $w$. However, in our new formulation, we have *multiple* addresses in $A$ that are considered nearby. We do not want to simply replace the contents of all of the nearby addresses with $w$. Why is this a bad idea? Because it's possible that some of those addresses are close to $x$ but also close to some other address $x'$, although $x$ and $x'$ may not themselves be close. We do not want to destroy information that was stored at previous times by just overwriting it.

Instead, we have a memory matrix $C$ of shape $M \times U$ that stores *counters*. For each element in $y_i$ of $y$, if the $y_i = 1$, then we will write $C_i$. For each bit $j$ from $1$ to $U$, if $w_j=1$, then we increment the $c_{ij}$. If $w_j$ is a $0$, then we decrement $C_{ij}$. The process of writing to memory becomes **distributed** across multiple addresses.

<img src="{{site.url}}/assets/sdm2.drawio.png" alt="">

How do we read from memory? Each row of $C$ contains $U$ counters, some of which may be positive and some of which may be negative. If a counter $C_{ij}$ is very positive, then mostly $1$s have been written to that counter. If a counter $C_{ij}$ is very negative, then mostly $0$s have been written to that counter. Let's say we have a memory address $x$ and we would like to read the data stored at that address. Like writing to memory, this address is almost certainly not represented in $A$. Therefore, we find all addresses in $A$ that are nearby to $x$ and create our vector $y$ representing which addresses are nearby. For every address in $A$ that is nearby, we take the corresponding row in $C$ and add them all together element-wise. We call this sum $s$, a vector of length $U$. For each bit position $j$ from $1$ to $U$, if more $1$s have been written to that position over every nearby address than $0$s, then $s_j$ will be positive. If more $0$s have been written to that position over every nearby address than $1$s, then $s_j$ will be negative. Therefore, we can threshold $s$ around $0$ to convert $s$ a binary vector $z$ representing the "average" binary vector stored across nearby addresses:
$$
z = \begin{cases} 1 & s > 0 \\ 0 & s \leq 0  \end{cases}
$$

<img src="{{site.url}}/assets/sdm3.drawio.png" alt="">

Using a vector $y$ to indicate whether or not an address in $A$ is nearby to $x$ is useful if we want to describe the operations of reading and writing to memory using matrix operations. However, I will not cover that approach here for several reasons:
1. It can become notationally heavy with the use of Hadamard products
2. The actual matrices used are sparse, so naive implementations can a long time to run (unless something like a `scipy.sparse.csr_matrix` is used)
3. We can take advantage of `numpy`'s broadcasting semantics and advanced indexing to perform equivalent operations quickly.

Below I show how one might implement SDM using `numpy`.

Let's use an object-oriented design in Python for this implementation. We will use one class to represent an instance of SDM. What kind of information do we need to specify an instance of SDM?


|Parameter|Description|
|:-:|:-:|
|`N`|Length of addresses in bits|
|`M`|Number of hard addresses|
|`U`|Length of data in bits|
|`d`|Hamming radius of addresses considered "near" for reading and writing.

Some things that may also be useful include a `seed`, since the addresses that we pick to generate $A$ are randomly selected from $\{ 0, 1 \}^N$ so being able to set the random seed ensures the same $A$ gets generated.

We will also need the class to have an address matrix $A$ and a counter matrix $C$. The following `__init__` method captures this information:

```python
def __init__(self, N, M, U, d, seed=None):
    self.N = N
    self.M = M
    self.d = d

    np.random.seed(seed)

    self.A = np.random.randint(low=0, high=2, size=(M, N), dtype=np.uint8)
    self.C = np.zeros((M, U), dtype=np.int8)
```

Technically speaking, the variables $N$ and $M$ do not need to be stored as parameters of the class, but storing them this way makes for easy accessibility and extensibility.

Next, we cover the operation of writing. In order to determine the Hamming distance between $w$ and a row of $A$, we can do

$$
x \text{ xor } A_i
$$

to get a vector of length $N$ which is $1$ where the bits of $x$ and $A_i$ differ, and is $0$ where they are the same. We can then sum the elements of this vector to get the Hamming distance. We can also take advantage of `numpy`'s broadasting semantics to vectorize this operation.

We can use `numpy.where` in place of a vector $y$ to generate and index set of rows where the Hamming distance is smaller than our predefined threshold $d$.

In order to add a $+1$ to rows of $C$ when bit $j$ of $w$ is a $1$ and add a $-1$ to rows of $C$ when bit $j$ of $w$ is a $0$, we can do
$$
C_i \gets C_i + 2w-1
$$
again taking advantage of `numpy`'s broadcasting semantics to vectorize this operation. 

Putting all this together gives us our `write` method:

```python
def write(self, x, w):
    h = np.logical_xor(x, self.A).sum(axis=1)
    y = np.where(h < self.d)
    self.C[y] += -1 + 2*w
```

Here, `h` is a vector of length $M$. Instead of being a vector of $1$s and $0$s, `y` here is just a vector of indices (rows) of `A` that are sufficiently close to `x`. We can then use `y` to index into `C` using advanced indexing and use `numpy`'s broadcasting semantics to modify each row of $C$ in the same way (adding or subtracting $1$).

Finally, we cover the operation of reading. We use the same code to generate our vector `y`. We can then easily sum along the columns of `C` indexed by `y` to generate `s`, and then threshold `s` to generate `z`:

```python
def read(self, x):
    h = np.logical_xor(x, self.A).sum(axis=1)
    y = np.where(h < self.d)
    s = self.C[y].sum(axis=0)
    z = (s > 0).astype(np.uint8)
return z
```

Putting everything together, we get the implementation of `sdmlib` that is available on PyPi and [github](github.com/avandekleut/sdmlib):

In [1]:
import scipy.stats as st
import numpy as np
from time import time

class Memory:
    def __init__(self, N, M, U, d, T=None, seed=None):
        if d is None:
            if T is None:
                raise ValueError(f'd and T cannot both be None.')
            else:
                # Optimality conditions found by Kanerva for uniformly
                # distributed addresses where z is a function that takes a
                # probability and returns a z score.
                d = N/2 + st.norm.ppf((2*M*T)**(-1/3.)) * (N/4)**(1/2)

        self.N = N
        self.M = M
        self.d = d

        np.random.seed(seed)

        self.A = np.random.randint(low=0, high=2, size=(M, N), dtype=np.uint8)
        self.C = np.zeros((M, U), dtype=np.int8)

    def write(self, x, w):
        h = np.logical_xor(x, self.A).sum(axis=1)
        y = np.where(h < self.d)
        self.C[y] += -1 + 2*w

    def read(self, x):
        h = np.logical_xor(x, self.A).sum(axis=1)
        y = np.where(h < self.d)
        s = self.C[y].sum(axis=0)
        z = (s > 0).astype(np.uint8)
        return z

Below is a toy example of generating random data and storing it in the memory, then retrieving it and comparing it to the original data:

In [2]:
import numpy as np
from sdmlib import Memory

N = 256
M = 1000
U = 256
T = 100

addresses =  np.random.randint(low=0, high=2, size=(T, N), dtype=np.uint8)
data      =  np.random.randint(low=0, high=2, size=(T, U), dtype=np.uint8)

mem = Memory(N=N, M=M, U=U, d=None, T=T)

for t in range(T):
    mem.write(addresses[t], data[t])

error = 0
for t in range(T):
    error += np.mean(data[t] != mem.read(addresses[t]))/T
print(f'Reconstruction error: {100*error:.2f}%')

Reconstruction error: 0.68%


# Final note

I have included an extra parameter called `T` in my implementation. This is a variable indicating the number of data to be written to memory. Sometimes this value is known in advance. If the bits of the addresses $x$ and data $w$ are identically and independently uniformly distributed, then there is an optimal value for the threshold $d$ that maximizes the *recall fidelity* of the memory (how closely retrieved values $z$ match written inputs $x$ once all the data has been written to memory).

Kanerva shows that the optimal probability of activating any particular hard address is given by 
$$
p = \frac{1}{\sqrt[3]{2MT}}
$$

The process randomly generating hard addresses in $A$ one bit at a time can be modelled as a Bernoulli process with $N$ trials, where we have equal probability of choosing a $0$ or a $1$. If we want to model the distribution of Hamming distances, we get a binomial distribution with a mean $\mu = N/2$ and variance $\sigma^2 = N/4$. For large $N$ (by assumption, $N$ is at least in the hundreds or thousands) we can make a normal approximation to the binomial distribution. 

We can then, for any given desired probability of activation $p$, derive the Hamming distance $d$ such that the probability of a random address matching another random address in $A$ is
$$
d = \frac{N}{2} + z \left( \frac{1}{\sqrt[3]{2MT}} \right) \sqrt{\frac{N}{4}}
$$

where
$$
z \left( \frac{1}{\sqrt[3]{2MT}} \right)
$$
is the $z$-score for the optimal probability derived by Kanerva.

For more reading, I highly recommend Kanerva's book on the topic.